# Preparation

In [47]:
from utils.data_prep import get_data, copy_and_apply_filter
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

In [48]:
df = get_data()

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12136 entries, 0 to 12135
Data columns (total 34 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Document Date              12136 non-null  datetime64[ns]
 1   Year/Month                 12136 non-null  period[M]     
 2   Year                       12136 non-null  int64         
 3   Month                      12136 non-null  int64         
 4   Supplier Delivery Date     12136 non-null  datetime64[ns]
 5   Delivery Date              12136 non-null  datetime64[ns]
 6   Company Code               12136 non-null  int64         
 7   Country                    12136 non-null  object        
 8   Purchasing Doc.            12136 non-null  int64         
 9   Item                       12136 non-null  int64         
 10  Purchasing Org.            12136 non-null  int64         
 11  Plant                      12136 non-null  int64         
 12  Supp

In [50]:
df.describe()

,Year,Month,Company Code,Purchasing Doc.,Item,Purchasing Org.,Plant,Supplier,Material,Net Price,Ordered Quantity,Delivered Quantity,Open Quantity,Delivery Deviation (Days),Deviation Cause,Net Value,Counter
count,12136.000000,12136.000000,12136.000000,1.213600e+04,12136.000000,12136.000000,12136.000000,1.213600e+04,1.213600e+04,12136.00000,12136.000000,12136.000000,12136.000000,12136.000000,12136.000000,12136.000000,12136.0
mean,2019.345831,5.870798,53.845089,8.202713e+09,2.263019,5198.619644,418.523401,5.856480e+06,5.971887e+07,59.11504,4788.483846,4785.820119,2.705339,0.049852,0.108438,8221.603334,1.0
std,0.475658,3.569839,0.492278,1.761377e+08,5.309245,1001.337060,1418.333240,1.257628e+07,1.877953e+07,310.12142,25327.854342,25313.664528,143.925618,1.032111,0.836305,38865.009635,0.0
min,2019.000000,1.000000,52.000000,1.500090e+09,1.000000,54.000000,51.000000,1.028900e+04,1.000348e+07,1.00000,0.250000,0.000000,-200.000000,-5.000000,0.000000,0.250000,1.0
25%,2019.000000,3.000000,54.000000,8.207604e+09,1.000000,5400.000000,51.000000,5.333800e+04,5.188012e+07,1.25000,13.000000,13.000000,0.000000,0.000000,0.000000,150.800000,1.0
50%,2019.000000,5.000000,54.000000,8.207793e+09,1.000000,5400.000000,51.000000,4.328630e+05,5.360295e+07,1.77000,200.000000,200.000000,0.000000,0.000000,0.000000,578.000000,1.0
75%,2020.000000,9.000000,54.000000,8.207941e+09,1.000000,5410.000000,51.000000,5.044180e+05,7.039588e+07,2.63000,1000.000000,1000.000000,0.000000,0.000000,0.000000,2110.000000,1.0
max,2020.000000,12.000000,54.000000,8.208073e+09,123.000000,5420.000000,9699.000000,3.432722e+07,9.494030e+07,6895.49000,590000.000000,590000.000000,10000.000000,75.000000,10.000000,612300.000000,1.0


In [51]:
df.nunique()

Document Date                 438
Year/Month                     18
Year                            2
Month                          12
Supplier Delivery Date        470
Delivery Date                 472
Company Code                    3
Country                         3
Purchasing Doc.              9926
Item                           63
Purchasing Org.                 8
Plant                           9
Supplier                      166
Supplier Name                 160
Postal Code                   157
Street                        163
City                          147
Supplier Country               12
Material                     3014
Material Short Text          3011
Material Group                164
Material Group Text           157
Order Unit                      6
Net Price                     297
Ordered Quantity              954
Delivered Quantity            956
Open Quantity                  23
Delivery Deviation (Days)      21
Deviation Indicator             4
Deviation Caus

In [52]:
df.head()

,Document Date,Year/Month,Year,Month,Supplier Delivery Date,Delivery Date,Company Code,Country,Purchasing Doc.,Item,Purchasing Org.,Plant,Supplier,Supplier Name,Postal Code,Street,City,Supplier Country,Material,Material Short Text,Material Group,Material Group Text,Order Unit,Net Price,Ordered Quantity,Delivered Quantity,Open Quantity,Delivery Deviation (Days),Deviation Indicator,Deviation Cause,Deviation Cause Text,Net Value,Local Currency,Counter
0,2019-02-28,2019-02,2019,2,2019-03-07,2019-03-07,52,AT,8207406178,1,5200,51,34295609,SEFREN CEBIN FRENCE,31770,12 AVENUE JEAN MERMOZ,COLOMIERS,FR,53286294,H379001-101 RAIL KIT UNDERBIN OVI,4017,Geraete ATA 25-3 Furnishing Pax Compartment,ST,672.0,10.0,7.0,3,0,in time,3,under-delivery,6720.0,EUR,1
1,2019-02-28,2019-02,2019,2,2019-03-07,2019-03-07,52,AT,8207406178,2,5200,51,34295609,SEFREN CEBIN FRENCE,31770,12 AVENUE JEAN MERMOZ,COLOMIERS,FR,53803787,H374001-105 MCD RAIL KIT OVI,4017,Geraete ATA 25-3 Furnishing Pax Compartment,ST,1816.2,10.0,7.0,3,0,in time,3,under-delivery,18162.0,EUR,1
2,2019-02-28,2019-02,2019,2,2019-03-07,2019-03-07,52,AT,8207406178,3,5200,51,34295609,SEFREN CEBIN FRENCE,31770,12 AVENUE JEAN MERMOZ,COLOMIERS,FR,53545566,H374001-103 MCD RAIL KIT OVI,4017,Geraete ATA 25-3 Furnishing Pax Compartment,ST,571.0,10.0,7.0,3,0,in time,3,under-delivery,5710.0,EUR,1
3,2019-02-28,2019-02,2019,2,2019-03-07,2019-03-07,52,AT,8207406178,4,5200,51,34295609,SEFREN CEBIN FRENCE,31770,12 AVENUE JEAN MERMOZ,COLOMIERS,FR,54078469,H374001-109 MCD RAIL KIT OVI,4017,Geraete ATA 25-3 Furnishing Pax Compartment,ST,2989.0,10.0,7.0,3,0,in time,3,under-delivery,29890.0,EUR,1
4,2019-02-28,2019-02,2019,2,2019-03-07,2019-03-07,52,AT,8207406178,5,5200,51,34295609,SEFREN CEBIN FRENCE,31770,12 AVENUE JEAN MERMOZ,COLOMIERS,FR,53286294,H379001-101 RAIL KIT UNDERBIN OVI,4017,Geraete ATA 25-3 Furnishing Pax Compartment,ST,742.0,13.0,10.0,3,0,in time,3,under-delivery,9646.0,EUR,1


# Ordered Spend

## Numeric Point Charts

In [53]:
def get_data_os_numeric_point_charts(df: pd.DataFrame) -> pd.DataFrame:
    df = df.groupby(['Year', 'Company Code', 'Purchasing Org.', 'Plant', 'Material Group']).agg({
        'Document Date':
        'count',
        'Net Value':
        'sum'
    }).reset_index().rename(columns={
        'Net Value': 'Ordered Spend',
        'Document Date': 'Number of Orders'
    })
    return df


In [54]:
def os_numeric_point_chart(df: pd.DataFrame,
                           last_year: bool = False,
                           company_code: str = None,
                           purchasing_org: str = None,
                           plant: str = None,
                           material_group: str = None) -> go.Figure:

    df = copy_and_apply_filter(df, company_code, purchasing_org, plant, material_group)
    df = df.groupby('Year').agg({'Number of Orders': 'count', 'Ordered Spend': 'sum'}).reset_index()

    if last_year:
        year = 2019
        mode = 'number'
        delta_os = None
        delta_no = None
    else:
        year = 2020
        mode = 'number+delta'
        delta_os = df.loc[df['Year'] == 2019, 'Ordered Spend'].iloc[0]
        delta_no = df.loc[df['Year'] == 2019, 'Number of Orders'].iloc[0]

    df = df.loc[df['Year'] == year]

    ordered_spend = df['Ordered Spend'].iloc[0]
    number_of_orders = df['Number of Orders'].iloc[0]

    fig = go.Figure()

    fig.add_trace(
        go.Indicator(mode=mode,
                     value=ordered_spend,
                     domain={
                         'x': [0, 0.45],
                         'y': [0, 1]
                     },
                     delta={
                         'reference': delta_os,
                         'relative': True
                     },
                     title='Ordered Spend'))
    fig.add_trace(
        go.Indicator(mode=mode,
                     value=number_of_orders,
                     domain={
                         'x': [0.55, 1],
                         'y': [0, 1]
                     },
                     delta={'reference': delta_no, 'relative': True},
                     title='Number of Orders'))

    fig.update_layout(title_text=year)
    return fig

In [55]:
df_point_charts = get_data_os_numeric_point_charts(df)

### This Year

In [56]:
os_numeric_point_chart(df_point_charts).show()

   Year  Number of Orders  Ordered Spend
0  2019               395   7.658801e+07
1  2020               225   2.318936e+07
   Year  Number of Orders  Ordered Spend
1  2020               225    23189364.21
76588013.848
395


### Last Year

In [57]:
os_numeric_point_chart(df_point_charts, True).show()

   Year  Number of Orders  Ordered Spend
0  2019               395   7.658801e+07
1  2020               225   2.318936e+07
   Year  Number of Orders  Ordered Spend
0  2019               395   7.658801e+07
None
None


## Line Charts

In [58]:
df_line_charts = df.groupby(['Year', 'Month']).agg({
    'Document Date': 'count',
    'Net Value': 'sum'
}).reset_index().rename(columns={
    'Net Value': 'Ordered Spend',
    'Document Date': 'Number of Orders'
})

In [59]:
def os_line_chart(df, number_of_orders=False):
    if number_of_orders:
        displayed = 'Number of Orders'
    else:
        displayed = 'Ordered Spend'

    df_this_year = df.loc[df['Year'] == 2020]
    df_last_year = df.loc[df['Year'] == 2019]
    
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=df_this_year['Month'], y=df_this_year[displayed], mode='lines+markers', name=2020))
    fig.add_trace(go.Scatter(x=df_last_year['Month'], y=df_last_year[displayed], mode='lines+markers', name=2019))
    
    fig.update_layout(title_text=displayed)

    return fig

### Ordered Spend by Month

In [60]:
os_line_chart(df_line_charts).show()

### Number of Orders by Month

In [61]:
os_line_chart(df_line_charts, True).show()

## Pie Charts

In [62]:
df_pie_charts = df.groupby(['Year', 'Purchasing Org.']).agg({
    'Document Date': 'count',
    'Net Value': 'sum'
}).reset_index().rename(columns={
    'Net Value': 'Ordered Spend',
    'Document Date': 'Number of Orders'
})

In [63]:
def os_pie_chart(df, number_of_orders = False):
    if number_of_orders:
        displayed = 'Number of Orders'
    else:
        displayed = 'Ordered Spend'

    df_this_year = df.loc[df['Year'] == 2020]
    df_last_year = df.loc[df['Year'] == 2019]

    fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'domain'}, {'type': 'domain'}]])

    # reverse
    fig.add_trace(go.Pie(labels=df_this_year['Purchasing Org.'], values=df_this_year[displayed], name=2020), 1, 1)
    fig.add_trace(go.Pie(labels=df_last_year['Purchasing Org.'], values=df_last_year[displayed], name=2019), 1, 2)
    fig.update_layout(title=displayed)
    return fig

### Ordered Spend by Organisation

In [64]:
os_pie_chart(df_pie_charts)

### Number of Orders by Organisation

In [65]:
os_pie_chart(df_pie_charts, True)

## Bar Chart

In [66]:
df_bar_charts = df.groupby(['Year', 'Supplier Name']).agg({
    'Document Date': 'count',
    'Net Value': 'sum'
}).reset_index().rename(columns={
    'Net Value': 'Ordered Spend',
    'Document Date': 'Number of Orders'
})
supplier_names = df_bar_charts.nlargest(10, ['Year', 'Ordered Spend'])['Supplier Name']
df_bar_charts = df_bar_charts.loc[df_bar_charts['Supplier Name'].isin(supplier_names)]

In [67]:
def os_bar_chart(df, number_of_orders=False):
    if number_of_orders:
        displayed = 'Number of Orders'
    else:
        displayed = 'Ordered Spend'

    df.sort_values(displayed, ascending=False, inplace=True)

    df_this_year = df.loc[df['Year'] == 2020]
    df_last_year = df.loc[df['Year'] == 2019]

    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=df_this_year['Supplier Name'],
        y=df_last_year[displayed],
        name=2020,
    ))
    fig.add_trace(go.Bar(
        x=df_last_year['Supplier Name'],
        y=df_last_year[displayed],
        name=2019,
    ))

    fig.update_layout(title=displayed, barmode='group', xaxis_tickangle=-45)
    return fig

### Ordered Spend Top 10 Suppliers

In [68]:
os_bar_chart(df_bar_charts).show()

### Number of Orders Top 10 Suppliers

In [69]:
os_bar_chart(df_bar_charts, True).show()

# Supplier Performance